# Data Analyst Assignment
> KreditBee - Collection Analytics

- The problem is a optimization problem to allot almost equal number of customers to all agents based on language prefrences.
- To solve it, pyomo package is used for defining variables, objective and constraints. CBC solver is employed (using neon server) to get the optimal solution.

In [1]:
import time
notebook_start_time = time.time()

In [2]:
#importing required packages
import pandas as pd
import numpy as np
import copy
from pyomo.environ import *
from pyomo.opt import SolverFactory
import itertools

In [3]:
#Reading the dataset
xls = pd.ExcelFile('Task_raw_data.xlsx')
agent = pd.read_excel(xls, 'Agent_details')
account = pd.read_excel(xls, 'Raw_Data')

In [4]:
xls2 = pd.ExcelFile('State & Language Details.xlsx')
lang_account = pd.read_excel(xls2, 'Sheet1')

In [5]:
agent.head()

,Employee_Name,Employee ID,Bucket,Product,Source,Hindi,Telugu,Kannada,Tamil
0,A,A11,NaN,MLA-15K,KB,1,0,1,0
1,B,B11,NaN,MLA-15K,KB,1,1,0,0
2,C,C11,NaN,MLA-15K,KB,0,1,1,1
3,d,D11,NaN,MLA-15K,KB,1,1,1,0
4,e,E11,NaN,MLA-15K,KB,1,0,0,0


In [6]:
agent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 9 columns):
Employee_Name    22 non-null object
Employee ID      22 non-null object
Bucket           0 non-null float64
Product          22 non-null object
Source           22 non-null object
Hindi            22 non-null int64
Telugu           22 non-null int64
Kannada          22 non-null int64
Tamil            22 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 1.7+ KB


In [7]:
account.head()

,Date,loanid,productName,age,city,state,cibil_score,Principal_due,Days_past_due,Source
0,2020-02-17,7553357,MLA-10K,33,Hyderabad,TELANGANA,589,4000,11,KB
1,2020-02-17,7587644,MLA-10K,33,Hyderabad,TELANGANA,628,4000,12,KB
2,2020-02-17,7651954,MLA-10K,25,Prakasam,ANDHRA PRADESH,628,4000,14,KB
3,2020-02-17,7671471,MLA-10K,23,Bengaluru,KARNATAKA,628,4000,8,KB
4,2020-02-17,7728829,MLA-10K,27,Hyderabad,TELANGANA,628,4000,8,KB


In [8]:
account.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2052 entries, 0 to 2051
Data columns (total 10 columns):
Date             2052 non-null datetime64[ns]
loanid           2052 non-null int64
productName      2052 non-null object
age              2052 non-null int64
city             2052 non-null object
state            2052 non-null object
cibil_score      2052 non-null int64
Principal_due    2052 non-null int64
Days_past_due    2052 non-null int64
Source           2052 non-null object
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 160.4+ KB


In [9]:
#Defining function for bucketing days_past_due
def bucket(x):
    if x <8:
        return '0-7'
    elif x <16:
        return '8-15'
    elif x <31:
        return '16-30'
    elif x <61:
        return '31-60'
    elif x >60:
        return '60+'
    else: return 'Other'

In [10]:
#Adding bucket column based on Days_past_due
account['bucket'] = account['Days_past_due'].apply(bucket)

In [11]:
account.groupby(['bucket']).size()

bucket
0-7      316
16-30    528
31-60    730
8-15     478
dtype: int64

In [12]:
#Adding language of customers to account table using lang_account reference table
account = account.merge(lang_account[['Language','State']], left_on = 'state', right_on = 'State').drop(['State'], axis =1)

In [13]:
account.groupby(['Language']).size()

Language
Hindi        1235
Kannada       200
Malayalam       8
Tamil         111
Telugu        498
dtype: int64

In [14]:
#Changing language of Malayalam customers to Tamil since we don't have any Malayalam speaking agents
account.loc[account['Language'] == 'Malayalam', 'Language'] = 'Tamil'

In [15]:
account.groupby(['Language']).size()

Language
Hindi      1235
Kannada     200
Tamil       119
Telugu      498
dtype: int64

In [16]:
#Transforming dataset to get languages of account in column form. It will be easier to match them with agents this way.

# generate binary values using get_dummies
dum_df = pd.get_dummies(account[['Language']])
# merge with main df account on key values 
account_2 = account.join(dum_df)
account_2

,Date,loanid,productName,age,city,state,cibil_score,Principal_due,Days_past_due,Source,bucket,Language,Language_Hindi,Language_Kannada,Language_Tamil,Language_Telugu
0,2020-02-17,7553357,MLA-10K,33,Hyderabad,TELANGANA,589,4000,11,KB,8-15,Telugu,0,0,0,1
1,2020-02-17,7587644,MLA-10K,33,Hyderabad,TELANGANA,628,4000,12,KB,8-15,Telugu,0,0,0,1
2,2020-02-17,7728829,MLA-10K,27,Hyderabad,TELANGANA,628,4000,8,KB,8-15,Telugu,0,0,0,1
3,2020-02-17,7962517,MLA-10K,27,Hyderabad,TELANGANA,628,4000,8,KB,8-15,Telugu,0,0,0,1
4,2020-02-17,8135897,MLA-10K,32,Hyderabad,TELANGANA,628,4000,9,KB,8-15,Telugu,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,2020-02-17,13515814,PK-5K,30,North Goa,GOA,607,4000,20,Kb,16-30,Hindi,1,0,0,0
2048,2020-02-17,16476882,PK-5K,28,Dimapur,NAGALAND,607,4000,24,Kb,16-30,Hindi,1,0,0,0
2049,2020-02-17,4065128,PK-5K,25,South Tripura,TRIPURA,675,3200,24,MZ,16-30,Hindi,1,0,0,0
2050,2020-02-17,6747198,PK-5K,27,West Tripura,TRIPURA,607,4000,8,KB,8-15,Hindi,1,0,0,0


In [17]:
#Filtering required columns
account_3 = account_2.iloc[:,[1,12,13,14,15]]

#Removing prefix from language columns
account_3.columns = account_3.columns.str.lstrip('Language_')

In [18]:
#Selecting relevant columns from agent table
agent_3 = agent.iloc[:,[1,5,6,7,8]]

#Changing column names to be same as account_3 table
agent_3 = agent_3[['Employee ID', 'Hindi', 'Kannada', 'Tamil', 'Telugu']]

In [19]:
print(agent_3.columns)
print(account_3.columns)

Index(['Employee ID', 'Hindi', 'Kannada', 'Tamil', 'Telugu'], dtype='object')
Index(['loanid', 'Hindi', 'Kannada', 'Tamil', 'Telugu'], dtype='object')


In [20]:
#Making a list of loanid and of employee ID as a identifier of account and agent. It will be used for mapping them later. 
#Customer list
cust = list(account_3['loanid'])

#Agent list
agt = list(agent_3['Employee ID'])

In [21]:
#Creating an availability matrix which gives potential customers which can be handled by an agent
availability = pd.DataFrame(np.ones((len(agt), len(cust))), index=agt, columns=cust)

#Populating availability matrix based on language
start_time = time.time()
for i in range(len(agt)):
    for j in range(len(cust)):
        if ((account_3.iloc[j,1] + agent_3.iloc[i,1]) == 2 or
        (account_3.iloc[j,2] + agent_3.iloc[i,2]) == 2 or
        (account_3.iloc[j,3] + agent_3.iloc[i,3]) == 2 or
        (account_3.iloc[j,4] + agent_3.iloc[i,4]) == 2):
            availability.iloc[i,j] = 1
        else:
            availability.iloc[i,j] = 0

end_time = time.time()
print(end_time - start_time)

availability = availability.astype(int)

8.407507181167603


In [22]:
#Transposing it to get in required format
availability = availability.transpose()

In [23]:
# Initialize pyomo concrete model
model2 = ConcreteModel()

# binary variables representing if a agent is scheduled to a customer
model2.works = Var(((customer, agent) for customer in cust for agent in agt),
                  within=Binary, initialize=availability.stack().to_dict())

In [24]:
# Define an objective function: For each agent, we want to minimize the total number of customers he attends
def obj_rule(m):
    for agent in agt:
        return 2*sum(m.works[customer, agent] for customer in cust)


# add objective function to the model. rule (pass function) or expr (pass expression directly)
model2.obj = Objective(rule=obj_rule, sense=minimize)

In [25]:
#Initializing a list to add constraints to
model2.constraints = ConstraintList()

In [26]:
#Constraint: one customer goes to only one employee only (not multiple employees per customer)
for customer in cust:
    model2.constraints.add(1 == sum(model2.works[customer, agent] for agent in agt))

In [27]:
#Constraint: no more than 94 customer per agent attended (average comes out to be 93.2)
for agent in agt:
    model2.constraints.add(
        94 >= sum(model2.works[customer, agent] for customer in cust)
    )

#Constraint: no less than 93 customer per agent attended
for agent in agt:
    model2.constraints.add(
        93 <= sum(model2.works[customer, agent] for customer in cust)
    )

In [28]:
#Constraint: Language of both customer and agent should be matched.
for (customer, agent) in list(itertools.product(cust, agt)):
    model2.constraints.add(model2.works[customer, agent] <= availability.loc[customer, agent])

In [29]:
#Solving the model using cbc solver on neos server
opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
results2 = solver_manager.solve(model2, opt=opt)

In [30]:
#Defining function to agent to customer mapping - model output
def get_work_table(works):
    table = {agent: [] for agent in agt}
    for agent in agt:
        for customer in cust:
            if works[customer, agent].value == 1:
                table[agent].append(customer)
    return table

alloted_cust = get_work_table(model2.works)

In [31]:
#Checking number of customers alloted per agent
x= []
for i in range (0,22):
    x.append((len(pd.DataFrame(alloted_cust.items())[1][i])))
    
print(x)

[93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 94, 94, 94, 93, 94, 94, 93, 94]


In [32]:
#Creating empty dataframe for customer to agent mapping
agt_allot = pd.DataFrame(columns = ['loanid', 'Employee ID'])

start_time = time.time()

#Appending values of customer, agent to dataframe if they are mapped according to model result
for (customer, agent) in list(itertools.product(cust, agt)):
    if model2.works[customer, agent].value == 1:
        agt_allot = agt_allot.append(pd.Series([customer, agent], index=agt_allot.columns), ignore_index=True)

end_time = time.time()
print(end_time - start_time)

2.7894959449768066


In [33]:
agt_allot.head()

,loanid,Employee ID
0,7553357,O11
1,7587644,M11
2,7728829,G11
3,7962517,D11
4,8135897,L11


In [34]:
#Verifying the results from model:

#Adding language of customer and agent to agt_allot:
agt_allot_lang_check = agt_allot.merge(account_3, on = 'loanid').merge(agent_3, on = 'Employee ID')

#appending values to an empty list as 1 if language of customer is a subset of language of alloted agent, and 10 if it is not
y = []
for i in range(len(agt_allot_lang_check)):
    if agt_allot_lang_check.iloc[i,2] <= agt_allot_lang_check.iloc[i,6]:
        y.append(1)
    else:
        y.append(10)

#Getting max of list to see if there is any value 10
max(y)

1

In [35]:
pd.DataFrame(alloted_cust.items()).to_csv('agent_to_customer.csv')

In [36]:
agt_allot.to_csv('customer_to_agent.csv')

In [37]:
notebook_end_time = time.time()
print('Notebook execution time =', round(notebook_end_time - notebook_start_time, 2), ' seconds')

Notebook execution time = 75.35  seconds
